# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [75]:
# import libraries
import nltk
nltk.download(['punkt', 'stopwords', 'wordnet'])

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier


[nltk_data] Downloading package punkt to /Users/saskia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/saskia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/saskia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [44]:
# !python -m pip install scikit-learn --upgrade

In [45]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('data_frame', engine)

In [46]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
X = df['message'].values
Y = df[df.columns[4:]]

In [48]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

### 2. Write a tokenization function to process your text data

In [49]:
def tokenize(text):
    
    urls = re.findall(url_regex, text)
    for url in urls:
        text = text.replace(url, 'urlplaceholder')
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    tokens = word_tokenize(text)
    
    token_wo_stop = []
    
    for token in tokens:
        if token not in stopwords.words('english'):
            token_wo_stop.append(token)
        
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok.lower().replace(' ', ''))
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [50]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('multi', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [52]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fdb07c5f940>)),
                ('tfidf', TfidfTransformer()),
                ('multi',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [53]:
y_pred = pipeline.predict(X_test)

In [54]:
accuracy = (y_pred == y_test).mean()
print('Accuracy:\n', accuracy)

Accuracy:
 related                   0.813244
request                   0.890601
offer                     0.987336
aid_related               0.765182
medical_help              0.914403
medical_products          0.944309
search_and_rescue         0.968721
security                  0.973451
military                  0.958346
child_alone               0.991608
water                     0.943699
food                      0.920812
shelter                   0.931034
clothing                  0.976808
money                     0.969789
missing_people            0.982606
refugees                  0.962008
death                     0.948886
other_aid                 0.863747
infrastructure_related    0.924474
transport                 0.952701
buildings                 0.945529
electricity               0.972994
tools                     0.986421
hospitals                 0.980012
shops                     0.986573
aid_centers               0.978181
other_infrastructure      0.947971
weather_r

In [55]:
target_names = df.columns[4:]

def classification_report_multi(ytest, ypred, targetnames):
    for i in range(0,len(targetnames)):
        print('Category {}'.format(targetnames[i]))
        report = classification_report(ytest[targetnames[i]], ypred[:,i])
        print(report)

In [56]:
classification_report_multi(y_test, y_pred, target_names)


Category related
              precision    recall  f1-score   support

           0       0.74      0.31      0.44      1537
           1       0.82      0.97      0.89      5017

    accuracy                           0.81      6554
   macro avg       0.78      0.64      0.66      6554
weighted avg       0.80      0.81      0.78      6554

Category request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5396
           1       0.87      0.45      0.59      1158

    accuracy                           0.89      6554
   macro avg       0.88      0.72      0.76      6554
weighted avg       0.89      0.89      0.88      6554

Category offer
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6478
           1       0.39      0.17      0.24        76

    accuracy                           0.99      6554
   macro avg       0.69      0.58      0.62      6554
weighted avg       0.98  

In [57]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7fdb07c5f940>)),
  ('tfidf', TfidfTransformer()),
  ('multi',
   MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7fdb07c5f940>),
 'tfidf': TfidfTransformer(),
 'multi': MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__subli

### 6. Improve your model
Use grid search to find better parameters. 

In [58]:
parameters = {"multi__estimator__bootstrap": (False, True),
             "tfidf__smooth_idf": (False, True)}

cv = GridSearchCV(pipeline, parameters)

In [59]:
cv.fit(X_train, y_train)
print("\nBest Parameters:", cv.best_params_)


Best Parameters: {'multi__estimator__bootstrap': False, 'tfidf__smooth_idf': True}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [60]:
y_pred_cv = cv.predict(X_test)

In [61]:
classification_report_multi(y_test, y_pred_cv, target_names)


Category related
              precision    recall  f1-score   support

           0       0.77      0.33      0.46      1537
           1       0.83      0.97      0.89      5017

    accuracy                           0.82      6554
   macro avg       0.80      0.65      0.68      6554
weighted avg       0.81      0.82      0.79      6554

Category request
              precision    recall  f1-score   support

           0       0.89      0.98      0.94      5396
           1       0.85      0.45      0.59      1158

    accuracy                           0.89      6554
   macro avg       0.87      0.72      0.76      6554
weighted avg       0.88      0.89      0.87      6554

Category offer
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6478
           1       0.41      0.20      0.27        76

    accuracy                           0.99      6554
   macro avg       0.70      0.60      0.63      6554
weighted avg       0.98  

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [63]:
pipeline_LSV = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('multi', MultiOutputClassifier(SGDClassifier()))])


In [64]:
pipeline_LSV.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fdb07c5f940>)),
                ('tfidf', TfidfTransformer()),
                ('multi', MultiOutputClassifier(estimator=SGDClassifier()))])

In [65]:
y_pred_LSV = pipeline_LSV.predict(X_test)

In [66]:
classification_report_multi(y_test, y_pred_LSV, target_names)

Category related
              precision    recall  f1-score   support

           0       0.74      0.43      0.54      1537
           1       0.84      0.95      0.90      5017

    accuracy                           0.83      6554
   macro avg       0.79      0.69      0.72      6554
weighted avg       0.82      0.83      0.81      6554

Category request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5396
           1       0.84      0.55      0.66      1158

    accuracy                           0.90      6554
   macro avg       0.87      0.76      0.80      6554
weighted avg       0.90      0.90      0.89      6554

Category offer
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6478
           1       0.00      0.00      0.00        76

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98  

/Users/saskia/opt/anaconda3/envs/disaster_response_pipeline/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/saskia/opt/anaconda3/envs/disaster_response_pipeline/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/saskia/opt/anaconda3/envs/disaster_response_pipeline/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter t

In [67]:
pipeline_LSV.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7fdb07c5f940>)),
  ('tfidf', TfidfTransformer()),
  ('multi', MultiOutputClassifier(estimator=SGDClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7fdb07c5f940>),
 'tfidf': TfidfTransformer(),
 'multi': MultiOutputClassifier(estimator=SGDClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf': Tru

In [68]:
parameters_LSV = {'multi__estimator__loss': ['hinge', 'perceptron']}

cv_LSV = GridSearchCV(pipeline_LSV, parameters_LSV)

In [72]:
cv_LSV.fit(X_train, y_train)
print("\nBest Parameters:", cv_LSV.best_params_)


Best Parameters: {'multi__estimator__loss': 'hinge'}


In [73]:
y_pred_cv_LSV = cv_LSV.predict(X_test)

In [74]:
classification_report_multi(y_test, y_pred_cv_LSV, target_names)

Category related
              precision    recall  f1-score   support

           0       0.75      0.41      0.53      1537
           1       0.84      0.96      0.90      5017

    accuracy                           0.83      6554
   macro avg       0.79      0.68      0.71      6554
weighted avg       0.82      0.83      0.81      6554

Category request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5396
           1       0.84      0.56      0.67      1158

    accuracy                           0.90      6554
   macro avg       0.87      0.77      0.81      6554
weighted avg       0.90      0.90      0.89      6554

Category offer
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6478
           1       0.00      0.00      0.00        76

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98  

/Users/saskia/opt/anaconda3/envs/disaster_response_pipeline/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/saskia/opt/anaconda3/envs/disaster_response_pipeline/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/saskia/opt/anaconda3/envs/disaster_response_pipeline/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter t

### 9. Export your model as a pickle file

In [78]:
dbfile = open('./models/trained_classifier', 'ab')

# source, destination
pickle.dump(y_pred_cv_LSV, dbfile)
dbfile.close()


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.